In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


In [0]:
df1 = pd.read_csv('train_genba.tsv', sep='\t')
df2 = pd.read_csv('train_goto.tsv', sep='\t')
df = pd.merge(df1, df2, on='pj_no', how='outer')

del df1
del df2

dftest1 = pd.read_csv('test_genba.tsv', sep='\t')
dftest2 = pd.read_csv('test_goto.tsv', sep='\t')
dftest = pd.merge(dftest1, dftest2, on='pj_no', how='outer')

del dftest1
del dftest2

df = df.reset_index(drop=False)
dftest = dftest.reset_index(drop=False)

# 出力用
dftest_id = dftest['id']

#欠損値
df = df.fillna(value=(-1))
df = df.replace('Nan', (-1))
dftest = dftest.fillna(value=(-1))
dftest = dftest.replace('Nan', (-1))

factorize = []

# string => float, int
for i, j in zip(df.columns, df.dtypes):
    if j == 'object':
        factorize.append(i)
    
for i in factorize:
    id_1, id_2 = df[i].factorize()
    id_3 = pd.DataFrame({i: id_1})
    df[i] = id_3
    dftest[i] = id_3

# X, Y
dfall = df.drop('keiyaku_pr', axis=1)
X = dfall.as_matrix()
Y = df['keiyaku_pr'].as_matrix()
XX = dftest.as_matrix()

scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
XX = scaler.fit_transform(XX)

maxlen = 158

X = np.array(X).reshape(len(X), maxlen, 1)
#train_Y = np.array(Y).reshape(len(Y), 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [0]:
# モデルの構築
n_in = 1
n_hidden = 20
n_out = 1


def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)


early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

model = Sequential()
model.add(SimpleRNN(n_hidden,
                    kernel_initializer=weight_variable,
                    input_shape=(maxlen, n_in)))
model.add(Dense(n_out, kernel_initializer=weight_variable))
model.add(Activation('linear'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mean_squared_error',
              optimizer=optimizer)

W0622 05:39:40.366324 140539616491392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0622 05:39:40.407939 140539616491392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0622 05:39:40.538094 140539616491392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
epochs = 500
batch_size = 10

model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping])

W0622 05:39:50.603646 140539616491392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0622 05:39:50.675530 140539616491392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0622 05:39:50.736873 140539616491392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Epoch 1/500
6461/6461 [==============================] - 53s 8ms/step - loss: 821008790817646.3750
Epoch 2/500
  20/6461 [..............................] - ETA: 48s - loss: 866376977743872.0000

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


6461/6461 [==============================] - 48s 7ms/step - loss: 821008000291639.5000
Epoch 3/500
6461/6461 [==============================] - 48s 7ms/step - loss: 821007236989333.6250
Epoch 4/500
6461/6461 [==============================] - 48s 7ms/step - loss: 821006475790347.1250
Epoch 5/500
6461/6461 [==============================] - 48s 7ms/step - loss: 821005725159889.8750
Epoch 6/500
6461/6461 [==============================] - 48s 7ms/step - loss: 821004973568657.5000
Epoch 7/500
6461/6461 [==============================] - 49s 8ms/step - loss: 821004216155645.1250
Epoch 8/500
6461/6461 [==============================] - 48s 7ms/step - loss: 821003460980980.0000
Epoch 9/500
6461/6461 [==============================] - 48s 7ms/step - loss: 821002710272622.2500
Epoch 10/500
6461/6461 [==============================] - 48s 7ms/step - loss: 821001956494976.5000
Epoch 11/500
6461/6461 [==============================] - 48s 7ms/step - loss: 821001201169703.2500
Epoch 12/500
6461/64

In [1]:
XX = np.array(XX).reshape(len(XX), maxlen, 1)

YY = model.predict(XX) * 1000000
df_pre = pd.DataFrame(YY)
df_pre.head()

NameError: ignored

In [2]:
df_pre = df_pre.astype(int)
df_pre = df_pre.round()
df_pre = pd.concat(objs=[dftest_id, df_pre], axis=1)
df_pre.head()

NameError: ignored

In [0]:
df_pre.to_csv('test_pre.tsv', sep='\t', index=False, header=False)